In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# загрузим наши данные и посмотрим информацию о датасете
df = pd.read_csv('sample-data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           500 non-null    int64 
 1   description  500 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [3]:
# посмотрим произвольный набор данных
df.sample(13)

,id,description
463,464,Daily grind - This fully padded shoulder bag k...
193,194,"Lithia skirt - After packing the quiver, who h..."
298,299,Active boy shorts - We've worn these versatile...
67,68,La surfer maria t-shirt - Pray for surf wearin...
98,99,O web belt - A social gadabout able to cross ...
415,416,Micro puff vest - Too many alpine shiver-fests...
141,142,Travel belt - The traveler's ace keeps your p...
11,12,"Baggies shorts - Even Baggies, our most popula..."
322,323,Vitaliti vee - The stretch and refined drape o...
464,465,Baby baggies apron dress - This lively dress k...


In [4]:
# Создадим вектор TF-DF
tfidf_vectorizer = TfidfVectorizer(binary=True, max_features=25000)
tfidf_embedings = tfidf_vectorizer.fit_transform(df['description'])

In [5]:
# посмотрим размер матрицы
tfidf_embedings.shape

(500, 4816)

In [6]:
# Расчет косинусного расстояния между товарами
cos_matrix = cosine_similarity(tfidf_embedings, tfidf_embedings)

In [7]:
# Определим пороговое значение для считывания похожих товаров
t = 0.7

In [8]:
# Найдем похожие товары
products = {}
for idx, row in df.iterrows():
    indices = cos_matrix[idx].argsort()[:-1]  # Исключаем сам товар
    items = [(cos_matrix[idx][i], df["id"][i]) for i in indices if cos_matrix[idx][i] > t]
    products[row["id"]] = items

In [9]:
# Выведем результат
for product_id, similar_items in products.items():
    print(f"Похожие товары для товара с id {product_id}:")
    for sim_score, sim_product_id in similar_items:
        print(f"Товар с id {sim_product_id}, Косинусное расстояние: {sim_score}")
    print("\n")

Похожие товары для товара с id 1:


Похожие товары для товара с id 2:


Похожие товары для товара с id 3:


Похожие товары для товара с id 4:
Товар с id 159, Косинусное расстояние: 0.9193644910666589


Похожие товары для товара с id 5:
Товар с id 308, Косинусное расстояние: 0.9679018559481283


Похожие товары для товара с id 6:


Похожие товары для товара с id 7:


Похожие товары для товара с id 8:
Товар с id 220, Косинусное расстояние: 0.9057903009954953


Похожие товары для товара с id 9:


Похожие товары для товара с id 10:


Похожие товары для товара с id 11:


Похожие товары для товара с id 12:


Похожие товары для товара с id 13:


Похожие товары для товара с id 14:


Похожие товары для товара с id 15:


Похожие товары для товара с id 16:


Похожие товары для товара с id 17:


Похожие товары для товара с id 18:


Похожие товары для товара с id 19:


Похожие товары для товара с id 20:
Товар с id 340, Косинусное расстояние: 0.7138317817116222


Похожие товары для товара с id 21:




## Используем метод Word2Vec

In [10]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import gensim
import gensim.downloader
from sklearn.preprocessing import normalize

In [11]:
# загрузим наши данные и посмотрим информацию о датасете
data = pd.read_csv('sample-data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           500 non-null    int64 
 1   description  500 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.9+ KB


In [24]:
# Обучение модели Word2Vec
sentences = [desc.split() for desc in data["description"]]
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=0)

In [13]:
# Создание векторов для товаров
def get_product_vector(desc, model):
    words = desc.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        # Усреднение векторов слов для получения вектора описания товара
        return np.mean(vectors, axis=0)  
    else:
        return None

In [14]:
data["vector"] = data["description"].apply(lambda x: get_product_vector(x, model))

In [28]:
# Определение порога значимости
t = 0.999  # Пороговое значение для считывания похожих товаров

In [29]:
# Поиск похожих товаров
products = {}
for idx, row in data.iterrows():
    similarities = []
    for i, other_row in data.iterrows():
        if idx != i:
            similarity = cosine_similarity([row["vector"]], [other_row["vector"]])[0][0]
            if similarity > t:
                similarities.append(other_row["id"])
    products[row["id"]] = similarities

In [30]:
# Вывод результатов
for product_id, similar_items in products.items():
    print(f"Похожие товары для товара с id {product_id}: {similar_items}")

Похожие товары для товара с id 1: [2, 3, 9, 14, 18, 19, 21, 25, 30, 32, 38, 39, 47, 60, 61, 62, 66, 70, 76, 106, 112, 121, 127, 132, 160, 164, 176, 180, 183, 190, 191, 192, 193, 194, 206, 218, 227, 228, 229, 234, 235, 245, 259, 264, 279, 287, 290, 295, 303, 309, 311, 314, 318, 321, 341, 345, 358, 362, 365, 369, 370, 374, 388, 411, 419, 451, 462, 463, 474, 487, 488, 494, 495, 499]
Похожие товары для товара с id 2: [1, 3, 9, 14, 18, 20, 21, 25, 27, 28, 30, 32, 38, 39, 47, 61, 62, 66, 70, 88, 102, 105, 106, 112, 117, 125, 132, 160, 161, 171, 172, 176, 180, 183, 190, 191, 192, 193, 194, 201, 206, 227, 228, 229, 234, 245, 259, 260, 264, 265, 279, 283, 287, 290, 295, 303, 309, 311, 314, 321, 324, 340, 341, 345, 348, 362, 365, 374, 378, 380, 388, 400, 411, 413, 419, 440, 443, 451, 452, 453, 462, 463, 474, 478, 487, 488, 494, 495, 499]
Похожие товары для товара с id 3: [1, 2, 9, 14, 18, 20, 21, 25, 27, 28, 30, 32, 34, 39, 47, 61, 62, 66, 70, 102, 105, 106, 112, 117, 132, 160, 161, 165, 171, 17

# Датасет слишком маленький для обучения, поэтому такой результат для модели Word2Vek